# VirtualiZarr + Kerchunk + Cubed

In [1]:
import cubed
import xarray as xr
import virtualizarr

In [2]:
! rm -rf combined* rechunked

### Open tutorial data

In [17]:
ds = xr.tutorial.open_dataset('air_temperature')
ds

<xarray.Dataset> Size: 31MB
Dimensions:  (lat: 25, time: 2920, lon: 53)
Coordinates:
  * lat      (lat) float32 100B 75.0 72.5 70.0 67.5 65.0 ... 22.5 20.0 17.5 15.0
  * lon      (lon) float32 212B 200.0 202.5 205.0 207.5 ... 325.0 327.5 330.0
  * time     (time) datetime64[ns] 23kB 2013-01-01 ... 2014-12-31T18:00:00
Data variables:
    air      (time, lat, lon) float64 31MB ...
Attributes:
    Conventions:  COARDS
    title:        4x daily NMC reanalysis (1948)
    description:  Data is from NMC initialized reanalysis\n(4x/day).  These a...
    platform:     Model
    references:   http://www.esrl.noaa.gov/psd/data/gridded/data.ncep.reanaly...

### Split into two NetCDF files

In [4]:
ds1 = ds.isel(time=slice(None, 1460))
ds2 = ds.isel(time=slice(1460, None))
ds1.to_netcdf('air1.nc')
ds2.to_netcdf('air2.nc')

/var/folders/8t/gslp67x10vgfjgv68qj92n0m0000gn/T/ipykernel_867/2127577491.py:3: SerializationWarning: saving variable air with floating point data as an integer dtype without any _FillValue to use for NaNs
  ds1.to_netcdf('air1.nc')
/var/folders/8t/gslp67x10vgfjgv68qj92n0m0000gn/T/ipykernel_867/2127577491.py:4: SerializationWarning: saving variable air with floating point data as an integer dtype without any _FillValue to use for NaNs
  ds2.to_netcdf('air2.nc')


### Combine two NetCDFs using VirtualiZarr

In [6]:
vds1 = virtualizarr.open_virtual_dataset('air1.nc', indexes={})
vds2 = virtualizarr.open_virtual_dataset('air2.nc', indexes={})
combined_vds = xr.concat([vds1, vds2], dim='time', coords='minimal', compat='override')

### Write references to Kerchunk

In [7]:
combined_vds.virtualize.to_kerchunk('combined.json', format='json')

### Open in Xarray using Cubed arrays

In [8]:
spec = cubed.Spec(allowed_mem="2GB", executor_name="single-threaded")
ds = xr.open_dataset(
    'combined.json',
    engine="kerchunk",
    chunked_array_type='cubed',
    from_array_kwargs={'spec': spec},
    chunks={}
)
ds

<xarray.Dataset> Size: 31MB
Dimensions:  (time: 2920, lat: 25, lon: 53)
Coordinates:
  * lat      (lat) float32 100B 75.0 72.5 70.0 67.5 65.0 ... 22.5 20.0 17.5 15.0
  * lon      (lon) float32 212B 200.0 202.5 205.0 207.5 ... 325.0 327.5 330.0
  * time     (time) datetime64[ns] 23kB 2013-01-01 ... 2014-12-31T18:00:00
Data variables:
    air      (time, lat, lon) float64 31MB cubed.Array<chunksize=(1460, 25, 53)>
Attributes:
    Conventions:  COARDS
    description:  Data is from NMC initialized reanalysis\n(4x/day).  These a...
    platform:     Model
    references:   http://www.esrl.noaa.gov/psd/data/gridded/data.ncep.reanaly...
    title:        4x daily NMC reanalysis (1948)

### Rechunk

In [15]:
# TODO: why do we need to pass in chunked_array_type and from_array_kwargs again?
ds2 = ds.chunk({"time": 2920, "lat": 5, "lon": 5}, chunked_array_type="cubed", from_array_kwargs={'spec': spec})
ds2

<xarray.Dataset> Size: 31MB
Dimensions:  (time: 2920, lat: 25, lon: 53)
Coordinates:
  * lat      (lat) float32 100B 75.0 72.5 70.0 67.5 65.0 ... 22.5 20.0 17.5 15.0
  * lon      (lon) float32 212B 200.0 202.5 205.0 207.5 ... 325.0 327.5 330.0
  * time     (time) datetime64[ns] 23kB 2013-01-01 ... 2014-12-31T18:00:00
Data variables:
    air      (time, lat, lon) float64 31MB cubed.Array<chunksize=(2920, 5, 5)>
Attributes:
    Conventions:  COARDS
    description:  Data is from NMC initialized reanalysis\n(4x/day).  These a...
    platform:     Model
    references:   http://www.esrl.noaa.gov/psd/data/gridded/data.ncep.reanaly...
    title:        4x daily NMC reanalysis (1948)

### Save rechunked arrays as Zarr

In [16]:
cubed.to_zarr(ds2["air"].data, "rechunked")

In [10]:
#ds.to_zarr("rechunked", safe_chunks=False, chunkmanager_store_kwargs={'spec': spec})